# Анализ и сегментация пользователей приложения "Ненужные вещи" на основе их поведения

Необходимо выделить группы пользователей мобильного приложения "Ненужные вещи" на основе их поведения, используя метрики: retention rate, время в приложении, частота действий и конверсия в просмотр контактов. 

**Ход работы:**

* Выгрузка данных

* Предобработка данных
    * Объединение данных
    * Проверка типов данных
    * Рассчет сессий
    * Проверка на пропуски
    * Проверка на дубликаты
* Анализ поведения пользователей
    * Расчет и анализ Retention Rate
    * Расчет и анализ времени в приложении
    * Расчет и анализ частоты действий пользователей
    * Расчет и анализ конверсии в просмотр контактов
* Сегментация пользователей
    * Деление пользователей на группы
    * Расчет Retention rate по группам пользователей
    * Расчет конверсии по группам пользователей
* Проверка гипотез
    * Различие конверсии в просмотр контактов между пользователями из Yandex и Google.

        • Нулевая гипотеза (H₀): Конверсия одинакова.

        • Альтернативная гипотеза (H₁): Конверсия различается.

    * Пользователи, которые кликают по рекомендованным объявлениям, имеют более высокую конверсию в просмотр контактов, чем те, кто этого не делает.

        • Нулевая гипотеза (H₀): Конверсия в просмотр контактов одинакова для пользователей, кликавших на рекомендации (tips_click), и тех, кто не кликал.
        
        • Альтернативная гипотеза (H₁): Конверсия в просмотр контактов выше у пользователей, кликавших на рекомендации.
* Выводы и рекомендации



**Описание данных**


Колонки в <code>/datasets/mobile_sources.csv</code>:

* <code>userId</code> — идентификатор пользователя,
* <code>source</code> — источник, с которого пользователь установил приложение.


Колонки в <code>/datasets/mobile_dataset.csv</code>:
* <code>event.time</code> — время совершения,
* <code>user.id</code> — идентификатор пользователя,
* <code>event.name</code> — действие пользователя.
    Виды действий:
    * <code>photos_show</code> — просмотрел фотографий в объявлении,
    * <code>advert_open</code> — открыл карточки объявления,
    * <code>tips_show</code> — увидел рекомендованные объявления,
    * <code>tips_click</code> — кликнул по рекомендованному объявлению,
    * <code>contacts_show</code> и <code>show_contacts</code> — посмотрел номер телефона,
    * <code>contacts_call</code> — позвонил по номеру из объявления,
    * <code>map</code> — открыл карту объявлений,
    * <code>search_1</code> — <code>search_7</code> — разные действия, связанные с поиском по сайту,
    * <code>favorites_add</code> — добавил объявление в избранное.



### Загрузка данных

In [ ]:
# Подключаем библиотеки

import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
import math as mth
import scipy.stats as st
from scipy.stats import chi2_contingency
import seaborn as sns
import os

In [ ]:
# Загружаем данные
pth1 = '/datasets/mobile_dataset.csv'
pth2 = 'https://code.s3.yandex.net/datasets/mobile_dataset.csv'

if os.path.exists(pth1):
    md = pd.read_csv(pth1, sep=',')
else:
    try:
        md = pd.read_csv(pth2, sep=',')
    except Exception as e:
        print(f"Ошибка загрузки файла: {e}")

md.info()

In [ ]:
pth3 = '/datasets/mobile_sourсes.csv'
pth4 = 'https://code.s3.yandex.net/datasets/mobile_sources.csv'

if os.path.exists(pth3):
        ms = pd.read_csv(pth3, sep=',')   
else:
    try:
        ms = pd.read_csv(pth4, sep=',')
    except Exception as e:
        print(f"Ошибка загрузки файла: {e}")
        
ms.info()

In [ ]:
# Приведем названия колонок к нижнему регистру и "змеиному" формату

md.columns = ['event_time', 'event_name', 'user_id']
ms.columns = ['user_id', 'source']

md.info()
ms.info()

### Предобработка данных

In [ ]:
# Объединим таблицы

df = pd.merge(md, ms, left_on='user_id', right_on='user_id')  

df.info()

Пропусков не замечено. Однако необходимо столбец event.time привести к формату времени.

In [ ]:
df.head()

In [ ]:
# Ищем дубликаты

df.duplicated().sum()

In [ ]:
# Приведем столбец event_time к формату времени

df['event_time'] = pd.to_datetime(df['event_time']).dt.round('ms')

df.info()

Округляем время до миллисекунд, чтобы избежать большого количества дубликатов, как если бы мы округляли до секунд.

In [ ]:
df.head()

In [ ]:
# Проверим еще раз на дубликаты и посмотрим на них если они есть

df.duplicated().sum()

In [ ]:
dub = df.duplicated(keep=False)
dub = dub[dub == True].index.to_list()

df.loc[df.index.isin(dub)]

По выведенной таблице можно сделать вывод, что в основном это связано с быстрым листанием ленты или фотографий. Таким образом и могут появляться повторные записи. Их не такое большое количество, так что можем их убрать.

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
# Проверим уникальные значения в столбце event_name

df.event_name.unique()

Видим что есть повторяющиеся значения показа контакта – contacts_show и show_contacts. Заменим 2-ое на 1-ое во всем датафрейме. Также есть события поиска в приложении под разными номерами. Объединим их в общее значение для более удобного анализа.

In [ ]:
search = ['search_1', 'search_2', 'search_3', 'search_4', 'search_5', 'search_6', 'search_7']

def contacts_rename(row):
    cell = row['event_name']
    if cell == 'show_contacts':
        return 'contacts_show'
    elif cell in search:
        return 'search'
    else:
        return cell
    
df['event_name'] = df.apply(contacts_rename, axis=1)

df.event_name.unique()

In [ ]:
# Выделяем сесси пользователей
session = (df.groupby('user_id')['event_time'].diff() > pd.Timedelta('30Min')).cumsum()

df['session_id'] = df.groupby(['user_id', session], sort=False).ngroup() + 1

df.head()

Выделяем сессии путем сортировки таблицы с логами событий и последующим добавлением уникального номера каждой, обозначив разрыв между 2 сессиями в 30 минут как условие для начала следующей сессии. 

***Выводы:***

В ходе данной части было сделано следующее:
* Датасеты были объединены в единую таблицу
* Наименования столбцов приведены в нижний регистр и к "змеиному" стилю
* В колонке event_time формат данных был изменен на временной 
* Проверили на дубликаты
* Изучены показатели колонки event_name, приведены к общим показатели "contacts_show" и "search"
* Выделены отдельные сессии для пользователей



Данные готовы к дальнейшему анализу.

### Анализ поведения пользователей

#### Расчет и анализ Retention Rate


Сначала подготовимся для расчетов. Необходимо составить таблицу с профилями пользователей, где будет содержаться информация из полей user_id, session_id, время начала и конца сессии, а так же источник, откуда пользователь пришел. 

In [ ]:
#Для его создания напишем функцию.

def get_sessions(df):
    
    session_s = (df.groupby(['session_id'])
             .agg({'event_time':'min'})
             .rename(columns={'event_time':'session_start'})
             .reset_index())

    session_e = (df.groupby(['session_id'])
             .agg({'event_time':'max'})
             .rename(columns={'event_time':'session_end'})
             .reset_index())
    
    sessions = df.merge(session_s,
                        on='session_id',
                        how='left', 
                        sort=True)
        
    sessions = sessions.merge(session_e, 
                              on='session_id', 
                              how='left',
                              sort=True)
    
    sessions = sessions[['user_id',
                         'session_id', 
                         'source',
                         'session_start', 
                         'session_end']]
    sessions = (sessions
                .drop_duplicates()
                .reset_index(drop=True))
    
    return sessions

sessions = get_sessions(df)
sessions.head()

In [ ]:
#Выведем таблицу с информацией о первых визитах пользователей на сайт.

profiles = (sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {'session_start': 'first',
                'source': 'first',}
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )
profiles



In [ ]:
profiles['dt'] = profiles['first_ts'].dt.date

def get_profiles(sessions):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'source': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )
    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['dt'] = profiles['dt'].astype('datetime64[ns]')
    profiles['month'] = profiles['dt'].values.astype('datetime64[M]')

    
    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'source'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )
    
    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        new_users[['dt', 'source']],
        on=['dt', 'source'],
        how='left',
    )
    
    return profiles

profiles = get_profiles(sessions)

Теперь создадим таблицу **Retention Rate**

In [ ]:
def get_retention(
    profiles, sessions, observation_date, horizon_days, ignore_horizon=False
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # рассчитываем удержание
    result_grouped = result_raw.pivot_table(
        index=['dt'], columns='lifetime', values='user_id', aggfunc='nunique'
    )
    cohort_sizes = (
        result_raw.groupby('dt')
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'cohort_size'})
    )
    result_grouped = cohort_sizes.merge(
        result_grouped, on='dt', how='left'
    ).fillna(0)
    result_grouped = result_grouped.div(result_grouped['cohort_size'], axis=0)

    # исключаем все лайфтаймы, превышающие горизонт анализа
    result_grouped = result_grouped[
        ['cohort_size'] + list(range(horizon_days))
    ]

    # восстанавливаем столбец с размерами когорт
    result_grouped['cohort_size'] = cohort_sizes

    # возвращаем таблицу удержания и сырые данные
    # сырые данные пригодятся, если нужно будет отыскать ошибку в расчётах
    return result_raw, result_grouped


In [ ]:
# Для постановки момента анализа, проверим последнюю доступную дату.

df_day=df.groupby(['session_id']).agg({'event_time':'max'})
             
df_day.tail()

Моментом анализа зададим дату – 3 ноября 2019 года, как мы видим по данным она последняя доступная нам; горизонт анализа будет равен 7-ми дням. Так мы рассмотрим еженедельную динамику коэффициента удержания пользователей.

In [ ]:
# задаём момент и горизонт анализа данных
observation_date = datetime(2019, 11, 3).date()
horizon_days = 7

# создаём опцию «игнорировать горизонт»
ignore_horizon = False

retention_raw, retention = get_retention(
    profiles, sessions, observation_date, horizon_days, ignore_horizon
)

In [ ]:
round(retention.loc[:, 6].mean() * 100, 2)

Среднее значение удержания на 7-й день – 6.03%

In [ ]:
# строим хитмэп без нулевого лайфтайма

plt.figure(figsize=(15, 6))  
sns.heatmap(
    retention.drop(columns=['cohort_size', 0]),  
    annot=True, 
    fmt='.2%',  
    cmap='Blues'
)
plt.title('Тепловая карта Retention Rate')
plt.xlabel('Дата первого визита')  
plt.ylabel('День с момента первого события') 
plt.show()


***Вывод***: 

Тепловая карта демонстрирует значительные колебания уровня удержания пользователей в зависимости от даты их первого визита.

  Например, когорта от 17 октября показывает нелинейную динамику: старт с 14.12%, резкий спад до 6.02% к 4-му дню и последующий рост до 9.41% к 6-му дню, что может быть связано с точечными маркетинговыми активностями или улучшением пользовательского опыта в эти дни.
  
  Высокие пики, такие как 16.00% у когорты от 12 октября и 14.99% от 26 октября, указывают на периоды эффективного привлечения или удержания аудитории. В то же время крайне низкие значения (например, 1.27% 28 октября) сигнализируют о проблемах в вовлечении или качестве трафика в отдельные периоды.

#### Расчет и анализ времени в приложении

In [ ]:
#Рассчитываем длительность сесссии в минутах

sessions['duration'] = (sessions['session_end'] - sessions['session_start'])
sessions['duration'] = (sessions['duration'].dt.total_seconds()//60).astype(float)

sessions.head()

Мы расчитали сколько по времени длятся каждая сессия. Теперь посмотрим максимальную и минимальную длительность.

In [ ]:
sessions_min=sessions['duration'].min()

sessions_max=sessions['duration'].max()

print('Минимальная длительность сессии: ', sessions_min)
print('Максимальная длительность сессии: ', sessions_max)

Мы видим что минимальная длительность сессии равна нулю, скорее всего это означает что пользователь зашел сделал одно какое-то действие и вышел. Проверим сколько таких сессий было и на какие действия приходились.

Однако так же вызывает вопрос максимальная сессия в 321 минуту. Провести 5 с половиной часов в приложении конечно можно, но лучше проверить для уточнения. 

In [ ]:
#Посмотрим нет ли резкого скачка по длительности сессии выведя 20 самых длинных

sessions=sessions.sort_values('duration', ascending=False)

sessions.head(20)

Удивительно, но судя по данным некоторые пользователи проводят часы за поиском нужного. Или, возможно, просто зависают в рекомендациях)


Теперь вернемся к вопросу с нулевыми сессиями.

In [ ]:
zero = round(sessions[sessions['duration'] == 0].shape[0] / sessions.shape[0] * 100, 2)

print('Процент нулевых сессий: ', zero)

Почти 30% сессий являются нулевыми. Посмотрим какие действия были выполнены за эти сессии.

In [ ]:
df_zero = df[
    df['session_id'].isin(sessions[sessions['duration'] == 0].session_id.tolist())].groupby('event_name')[['session_id']].count().sort_values(by='session_id', ascending=False)
            
df_zero

Наиболее вероятными точками завершения пользовательских сессий стали *просмотр рекомендаций*, *изучение фотографий* в карточке объявления и *выполнение поисковых действий* — эти этапы чаще всего предшествовали закрытию приложения.

In [ ]:
sessions.loc[sessions['duration'] == 0, 'duration'] = np.nan

sessions[['duration']].describe().round(2)

Средняя длительность сессии составляет 17,45 минут, при этом медиана (11 минут) указывает на преобладание коротких взаимодействий: 25% сессий не превышают 4 минут, а 75% — 23 минут. Максимальная зафиксированная сессия длилась 321 минуту (~5,35 часов), что наряду с высоким стандартным отклонением (21,30 минуты) подтверждает значительную вариативность данных: большинство пользователей быстро завершают сессии, но присутствуют редкие аномалии, требующие дополнительной проверки (например, корректности измерения времени).

In [ ]:
plt.figure(figsize=(15, 7))
sns.histplot(
    sessions['duration'].dropna(), 
    bins=90,
    color=sns.color_palette("rocket", 1)[0]
)
plt.show()

Средняя длительность сессий пользователей не превышает 52 минут, при этом медиана (12,2 минуты) указывает на равное распределение данных: 50% значений лежат выше и ниже этой границы. Сессии длительностью свыше 100 минут встречаются крайне редко (частота ≈1 случай), что подтверждает наличие аномалий. Гистограмма демонстрирует резкий спад частоты после 52 минут, выделяя зону выбросов за пределами «нормального» диапазона.

Многочасовые сессии можно объяснить тем что человек может выйти и вернуться в приложении меньше чем за изначальный интервал в 30 минут, в этом случае у человека получается одна большая сессия.

In [ ]:
sessions['date'] = sessions['session_start'].dt.date

day = sessions.pivot_table(index='date', values='duration', aggfunc=['sum', 'mean']).reset_index()
day.columns = day.columns.droplevel(1)
day['date'] = pd.to_datetime(day['date'])

day['seven_day_sum'] = day['sum'].rolling(7).mean()
day['seven_day_mean'] = day['mean'].rolling(7).mean()

fig = px.line(day, 
             x='date',
             y=['sum', 'seven_day_sum'], markers=True,
             labels={'sum':'Сумма минут', 'date':'Date'},
             title='Суммарно время в приложении по дням, минуты',
             color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

Для корректной интерпретации графика необходимо учитывать среднее время, проведенное пользователями в приложении за день, так как рост общего времени использования может быть связан как с увеличением активности, так и с притоком новых пользователей. Для анализа динамики построим график среднесуточного времени использования приложения за 4 недели с ежедневной детализацией.

In [ ]:
fig = px.line(day, 
             x='date',
             y=['mean','seven_day_mean'], markers=True,
             labels={'mean':'Минут в среднем', 'dt':'Date'},
             title='Среднее время в приложении по дням, минуты',
             color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()


Наблюдается общая тенденция к снижению среднего времени сессий: с 21 минуты (7 октября) до 18 минут (3 ноября). При этом данные демонстрируют выраженную волатильность: 9 октября зафиксирован резкий спад до минимума в 15 минуты (-27% за день), а уже 13 октября — скачок роста на 22.5% (с 15 до 21 минуты).

За весь анализируемый период зафиксирован рост общего времени использования приложения: с 4010 минут до 5642 минуты в день, что указывает на положительную динамику вовлеченности. Однако на графике наблюдаются выраженные спады активности — наиболее значительные провалы произошли 12 октября 2019 года и 2 ноября 2019 года, требующие отдельного изучения причин (технические сбои, сезонные факторы или изменения в контенте).

***Вывод:***

Рост общего времени использования приложения на 46% (с 4325 до 6310 минут/день) свидетельствует о успешном привлечении новых пользователей, которые обращаются к сервису в моменты свободного времени. Однако параллельное снижение средней длительности сессии на 5.6% (с 23.1 до 21.8 минут) указывает на ухудшение вовлеченности: новые пользователи ограничиваются краткими взаимодействиями (25% сессий ≤4 минут), а резкие спады активности (12.10 и 2.11) и экстремальная волатильность (-27% за день 9.10) демонстрируют нестабильность метрик.

#### Расчет и анализ частоты действий пользователей

Для начала посмотрим процент действий по каждому событию и построим наглядный график.

In [ ]:
freq = (df.groupby('event_name')['event_time']
              .count()
              .reset_index()
              .sort_values(by='event_time',
                           ascending=False))
freq = freq.rename(columns={'event_time':'freq'})
freq['percent'] = round(freq['freq'] / freq['freq'].sum() * 100, 2)

freq

In [ ]:
fig = px.bar(freq, 
             x='freq',
             y='event_name',
             text='percent',
             title='Распределение частоты событий',
             color_discrete_sequence=px.colors.qualitative.Dark24)

fig.update_traces(textposition='outside')

fig.update_layout(
    title_text='Распределение частоты событий',
    xaxis_title='Частота', 
    yaxis_title='События',
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()

***Вывод:***

Более половины (54%) всех действий приходится на показ рекомендаций, однако крайне низкий процент кликов по ним (всего 1.1%) указывает на неэффективность алгоритмов и необходимость их оптимизации. При этом звонки по контактам (0.73%) — самое редкое, но наиболее значимое событие, отражающее максимальный интерес к товару/услуге и высокий потенциал конверсии.

Стандартные действия демонстрируют умеренную активность: поиск (9.15%), просмотр фотографий (13.48%), открытие карты (5.22%) и показ контактов (6.09%). Особого внимания заслуживает низкая частота добавления в избранное (1.91%) — это свидетельствует о недостаточной мотивации пользователей к сохранению карточек, несмотря на относительно высокий процент открытия объявлений (8.31%).

#### Расчет и анализ конверсии в просмотр контактов

Начнем с просмотра общей конверсии за весь период, представленный в данных.

In [ ]:
conversion = (df[df['event_name'] == 'contacts_show']['user_id'].nunique() / df['user_id'].nunique() * 100)
conversion = round(conversion, 2)
print(f"Конверсия в целевое действие за весь период: {conversion}%")

Конверсия в просмотр контактов за анализируемый период составила 22.85%, это означает что примерно каждый пятый пользователь проявил достаточный интерес, чтобы перейти к просмотру контактной информации.

Посмотрим теперь тенденции в разрезе недель и дней.

In [ ]:
# Для начала напишем функцию, которая посчитает нам эти данные

def calculate_conversion_dynamics(df, date_col='event_time', target_event='contacts_show'):
    
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df[date_col] = df[date_col].dt.date
    df = df.sort_values(by=date_col)

    # Расчет по дням
    daily_users = df.groupby(date_col)['user_id'].nunique().reset_index(name='total_users')
    daily_target = (
        df[df['event_name'] == target_event]
        .groupby(date_col)['user_id'].nunique()
        .reset_index(name='target_users')
    )
    daily_conversion = daily_users.merge(daily_target, on=date_col, how='left')
    daily_conversion['conversion_rate'] = (
        daily_conversion['target_users'] / daily_conversion['total_users'] * 100
    ).round(2)
    daily_conversion.rename(columns={date_col: 'date'}, inplace=True)

    daily_conversion = daily_conversion.tail(7)

    # Расчет по неделям
    df['week'] = pd.to_datetime(df[date_col]).dt.to_period('W').dt.to_timestamp()
    df['week'] = df['week'].dt.date
    weekly_users = df.groupby('week')['user_id'].nunique().reset_index(name='total_users')
    weekly_target = (
        df[df['event_name'] == target_event]
        .groupby('week')['user_id'].nunique()
        .reset_index(name='target_users')
    )
    weekly_conversion = weekly_users.merge(weekly_target, on='week', how='left')
    weekly_conversion['conversion_rate'] = (
        weekly_conversion['target_users'] / weekly_conversion['total_users'] * 100
    ).round(2)
    weekly_conversion.rename(columns={'week': 'week_start'}, inplace=True)

    return daily_conversion, weekly_conversion


In [ ]:
daily_conv, weekly_conv = calculate_conversion_dynamics(df)

In [ ]:
# Конверсия за последние 7 дней
daily_conv

In [ ]:
daily_conv_last_7_days = daily_conv.tail(7)

plt.figure(figsize=(10, 20))

# Дневная конверсия
plt.subplot(2, 1, 1)
sns.barplot(
    x='conversion_rate',
    y='date',
    data=daily_conv_last_7_days,
    palette='coolwarm'
)
plt.title('Дневная конверсия в просмотр контактов')
plt.xlabel('Конверсия (%)')
plt.ylabel('Дата')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Конверсия по неделям
weekly_conv

In [ ]:
weekly_conv['week_start'] = pd.to_datetime(weekly_conv['week_start']).dt.date

plt.figure(figsize=(20, 20))

plt.subplot(2, 1, 2)
sns.barplot(
    y='week_start',  # Меняем оси местами
    x='conversion_rate',
    data=weekly_conv,
    palette='viridis'
)
plt.title('Недельная конверсия в просмотр контактов')
plt.xlabel('Конверсия (%)')
plt.ylabel('Начало недели')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

***Вывод:***

За последнюю неделю октября – начало ноября зафиксирован устойчивый рост дневной конверсии в просмотр контактов: с минимума 16.95% (28 октября) до пика 22.58% (1 ноября) с последующей стабилизацией на уровне ~20.7%. Параллельно наблюдалось снижение общего числа пользователей (-16% к 3 ноября), что подчеркивает повышение качества вовлечения оставшейся аудитории.

В среднесрочной перспективе (октябрь) конверсия демонстрирует позитивный тренд: еженедельный рост с 19.29% до 21.05% (+1.76 п.п.) на фоне увеличения трафика на 36% (1130 → 1546 пользователей). Это свидетельствует об эффективности платформы при масштабировании, но требует мониторинга влияния роста аудитории на глубину взаимодействия.

#### Выводы:

Удержание пользователей слабое: на 7-й день остаётся только 6% (при норме 20-40%). Это серьёзная проблема. Хотя общее время в приложении растёт за счёт новых пользователей, среднее время на человека почти не меняется. Особенно тревожит, что 29% сессий длятся 0 секунд — чаще всего люди закрывают приложение после просмотра рекомендаций (54%), фото (13.5%) или поиска (9.2%). Эти функции явно требуют доработки.

Конверсия (просмотр контактов) постепенно растёт: за месяц увеличилась с 19.3% до 21%. Лучшие дни для конверсии — вторник и пятница. В выходные пользователи действуют целенаправленнее, а в будни — больше лишних действий. 

### Сегментация пользователей

#### Деление пользователей на группы


Разделим пользователей на три группы в зависимости от того, сколько раз они запускали приложение (количество сессий). Такое деление позволит оценить лояльность, качество трафика и выявить проблемные точки в пользовательском пути. 

Границы сегментации выбраны на основе данных: 
* 1 сессия — критический порог первого впечатления, 
* 2-3 сессии — зона формирования привычки, 
* 4+ сессий — стадия лояльности. 

Ожидается, что активные пользователи будут демонстрировать более высокую конверсию и удержание по сравнению с умеренными и однократными. Это поможет превратить абстрактную аудиторию в управляемые сегменты, для каждого из которых можно разработать конкретные действия: триггеры возврата для однократных, стимулы для перехода к 4+ сессиям для умеренных и программы удержания для активных пользователей.


In [ ]:
user_sessions = sessions.groupby('user_id')['session_id'].nunique().reset_index()
user_sessions['segment'] = pd.cut(user_sessions['session_id'],
                                  bins=[0, 1, 3, float('inf')],
                                  labels=['Однократные', 'Умеренные', 'Активные'])

user_sessions

In [ ]:
segment_count = user_sessions['segment'].value_counts()

segment_count

In [ ]:
segment_counts = user_sessions['segment'].value_counts()
plt.figure(figsize=(10,6))
sns.barplot(x=segment_counts.values, y=segment_counts.index, palette="viridis")
plt.title('Распределение пользователей по сегментам')
plt.xlabel('Количество пользователей')
plt.show()

***Вывод:***

Больше половины пользователей (53.6%, 2300 человек) открыли приложение всего один раз и больше не возвращались. Это серьёзная проблема — скорее всего, им что-то не понравилось сразу: может быть сложный интерфейс, ошибки или просто не нашли то, что искали. Ещё треть (31.2%, 1340 человек) заходили 2-3 раза — они проявили интерес, но не стали постоянными. И только 15% (653 человека) активно используют приложение — это наши лояльные пользователи.

#### Расчет Retention rate по группам пользователей

Тепер проанализируем поведение выделенных групп по известным нам метрикам.

In [ ]:
retention_data = []
for segment in ['Однократные', 'Умеренные', 'Активные']:
    segment_users = user_sessions[user_sessions['segment'] == segment]['user_id']
    
    # Фильтруем сессии пользователей сегмента
    segment_sessions = sessions[sessions['user_id'].isin(segment_users)]
    
    # Находим первую дату сессии для каждого пользователя
    first_sessions = segment_sessions.groupby('user_id')['session_start'].min().reset_index()
    first_sessions.columns = ['user_id', 'first_session']
    
    # Находим сессии через 7 дней после первой
    seventh_day_sessions = segment_sessions.merge(first_sessions, on='user_id')
    seventh_day_sessions['day_diff'] = (seventh_day_sessions['session_start'] - seventh_day_sessions['first_session']).dt.days
    retained_users = seventh_day_sessions[seventh_day_sessions['day_diff'] == 7]['user_id'].nunique()
    
    # Рассчитываем процент
    retention_rate = (retained_users / len(segment_users)) * 100
    retention_data.append({'segment': segment, 'retention_7d': retention_rate})

retention_df = pd.DataFrame(retention_data)

retention_df

In [ ]:
fig = px.bar(retention_df, x='segment', y='retention_7d', 
             title='Retention Rate по сегментам',
             text='retention_7d',
             color='segment', 
    color_discrete_sequence=px.colors.qualitative.Pastel
             )
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

Построим тепловую карту удержания для наглядности анализа.

In [ ]:
observation_date = pd.to_datetime(datetime(2019, 10, 28).date())  # Изменяем дату на 28 октября 2019
horizon_days = 7

ignore_horizon = False

first_sessions = sessions.groupby('user_id')['session_start'].min().reset_index()
first_sessions.columns = ['user_id', 'first_session']

sessions_enriched = sessions_with_segments.merge(first_sessions, on='user_id')

sessions_enriched['days_since_first'] = (sessions_enriched['session_start'] - sessions_enriched['first_session']).dt.days

if not ignore_horizon:
    start_date = observation_date - timedelta(days=horizon_days)
    sessions_enriched = sessions_enriched[
        (sessions_enriched['first_session'] >= start_date) &
        (sessions_enriched['first_session'] <= observation_date)
    ]

sessions_enriched['first_session_date'] = sessions_enriched['first_session'].dt.strftime('%Y-%m-%d')

# Функция для расчета Retention Rate
def calculate_retention(df, segment_name):
    segment_data = df[df['segment'] == segment_name]

    cohort_pivot = segment_data.pivot_table(
        index='first_session_date', 
        columns='days_since_first',
        values='user_id',
        aggfunc=pd.Series.nunique,
        fill_value=0
    )

    cohort_size = cohort_pivot[0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis=0) * 100

    retention_matrix = retention_matrix.loc[:, 0:horizon_days]

    return retention_matrix

segments = ['Однократные', 'Умеренные', 'Активные']

plt.figure(figsize=(15, 20))  

for i, segment in enumerate(segments, 1):
    retention_matrix = calculate_retention(sessions_enriched, segment)

    plt.subplot(3, 1, i) 
    sns.heatmap(
        retention_matrix,
        annot=True,
        fmt='.1f',
        cmap='Blues',
        vmin=0,
        vmax=100,
        cbar_kws={'label': 'Retention Rate (%)'},
        linewidths=.5
    )
    plt.title(f'{segment} пользователи')
    plt.xlabel('Дни с первой сессии')
    plt.ylabel('Дата первой сессии')

plt.tight_layout()
plt.suptitle('Динамика удержания по сегментам пользователей', y=1.02, fontsize=16)
plt.show()

***Вывод:*** Удержание пользователей на 7-й день демонстрирует прямую зависимость от активности:

* Активные пользователи (22.4%) возвращаются в 5 раз чаще умеренных и формируют лояльное ядро.

* Умеренные пользователи (4.5%) имеют потенциал для роста через стимулирование повторных сессий.

* Однократные пользователи (0%) полностью теряются, что подтверждает критическую необходимость улучшения первого опыта.

Тепловые карты так же показывают, что однократные пользователи не возвращаются после первого дня, умеренные пользователи демонстрируют постепенное снижение удержания, а активные пользователи сохраняют наивысший уровень вовлеченности и удержания на протяжении всех семи дней. Это указывает на то, что активные пользователи являются наиболее лояльными и стабильными.

#### Расчет конверсии по группам пользователей

Теперь посмотрим на конверсию в целевое действие.

In [ ]:
merged_data = sessions.merge(df, on=['user_id', 'session_id'], how='left')

user_segments = sessions.groupby('user_id')['session_id'].nunique().reset_index()
user_segments['segment'] = pd.cut(user_segments['session_id'],
                                  bins=[0, 1, 3, float('inf')],
                                  labels=['Однократные', 'Умеренные', 'Активные'])
merged_data = merged_data.merge(user_segments[['user_id', 'segment']], on='user_id')

In [ ]:
conversion_data = []
for segment in ['Однократные', 'Умеренные', 'Активные']:
    segment_users = merged_data[merged_data['segment'] == segment]['user_id'].nunique()
    converted_users = merged_data[(merged_data['segment'] == segment) & 
                                 (merged_data['event_name'] == 'contacts_show')]['user_id'].nunique()
    conversion_rate = (converted_users / segment_users) * 100
    conversion_data.append({'segment': segment, 'conversion_rate': conversion_rate})

conversion_df = pd.DataFrame(conversion_data)

conversion_df

In [ ]:
fig = px.bar(conversion_df, x='segment', y='conversion_rate', 
             title='Конверсия по сегментам',
             text='conversion_rate',
             color='segment', 
    color_discrete_sequence=px.colors.qualitative.Pastel
             )
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.show()

***Вывод:***
Конверсия в целевое действие (просмотр контактов) так же демонстрирует выраженную зависимость от активности пользователей:

* Однократные пользователи показывают минимальную конверсию (14.8%), что подтверждает поверхностное взаимодействие с приложением.

* Умеренные пользователи (26.4%) конвертируются в 1.8 раза лучше однократных, отражая рост интереса при повторных посещениях.

* Активные пользователи достигают пиковой конверсии (44%), что в 3 раза выше однократных и подчеркивает ценность лояльного ядра.

#### Выводы:

Анализ выявил прямую зависимость между активностью пользователей и ключевыми метриками: активные пользователи (4+ сессии) демонстрируют максимальные показатели удержания (22.4% на 7-й день) и конверсии (44%), что в 5 раз выше удержания и в 3 раза выше конверсии умеренных пользователей (2-3 сессии). При этом однократные пользователи (53.6% аудитории) практически не возвращаются (0% retention), несмотря на базовую конверсию 14.8%, что указывает на ценность продукта, но критические барьеры первого опыта.

Наибольший потенциал роста — в переводе однократных пользователей в умеренные. Уже после второй сессии конверсия вырастет на 78% (с 14.8% до 26.4%), и появится удержание (4.5%).

### Проверка гипотез

#### Различие конверсии в просмотр контактов между пользователями из Yandex и Google.

Будем проверять следующую гипотезу: "Отличается ли конверсия просмотра контактов между пользователями из Yandex и Google".

* Нулевая гипотеза (H₀): Конверсия одинакова.
* Альтернативная гипотеза (H₁): Конверсия различается.

In [ ]:
yandex_users = df[df['source'] == 'yandex']['user_id'].unique()
google_users = df[df['source'] == 'google']['user_id'].unique()

def calculate_conversion(users):
    converted = df[(df['user_id'].isin(users)) & 
                  (df['event_name'] == 'contacts_show')]['user_id'].nunique()
    return converted, len(users)

yandex_converted, yandex_total = calculate_conversion(yandex_users)
google_converted, google_total = calculate_conversion(google_users)

# Создание таблицы сопряженности
contingency_table = pd.DataFrame({
    'Converted': [yandex_converted, google_converted],
    'Not Converted': [yandex_total - yandex_converted, google_total - google_converted]
}, index=['Yandex', 'Google'])

# Проверка гипотезы с помощью теста хи-квадрат
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"Конверсия Yandex: {yandex_converted}/{yandex_total} ({yandex_converted/yandex_total*100:.2f}%)")
print(f"Конверсия Google: {google_converted}/{google_total} ({google_converted/google_total*100:.2f}%)")
print(f"p-value: {p_value:.5f}")

if p_value < 0.05:
    print("Отвергаем H₀: Конверсия значимо различается")
else:
    print("Не отвергаем H₀: Конверсия не различается")


***Вывод:***
По результатам проверки конвверсия в целевое действие не зависит от ресурса приклекшего пользователя. Нулевая гипотеза не отвергается.

#### Пользователи, которые кликают по рекомендованным объявлениям, имеют более высокую конверсию в просмотр контактов, чем те, кто этого не делает.

Следующая гипотеза для проверки: "Отличается ли конверсия просмотра контактов между пользователями которые кликают по рекомендованным объявлениям и теми кто этого не делает".

* Нулевая гипотеза (H₀): Конверсия в просмотр контактов одинакова для пользователей, кликавших на рекомендации (tips_click), и тех, кто не кликал.
* Альтернативная гипотеза (H₁): Конверсия в просмотр контактов выше у пользователей, кликавших на рекомендации.

In [ ]:
tips_click_users = df[df['event_name'] == 'tips_click']['user_id'].unique()
non_click_users = df[~df['user_id'].isin(tips_click_users)]['user_id'].unique()

# Подсчет конверсии
click_converted, click_total = calculate_conversion(tips_click_users)
non_click_converted, non_click_total = calculate_conversion(non_click_users)

# Создание таблицы сопряженности
contingency_table = pd.DataFrame({
    'Converted': [click_converted, non_click_converted],
    'Not Converted': [click_total - click_converted, non_click_total - non_click_converted]
}, index=['Click', 'No Click'])

# Проверка гипотезы с помощью z-теста пропорций
from statsmodels.stats.proportion import proportions_ztest

count = [click_converted, non_click_converted]
nobs = [click_total, non_click_total]
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')

print(f"Конверсия кликавших: {click_converted}/{click_total} ({click_converted/click_total*100:.2f}%)")
print(f"Конверсия не кликавших: {non_click_converted}/{non_click_total} ({non_click_converted/non_click_total*100:.2f}%)")
print(f"p-value: {p_value:.5f}")

if p_value < 0.05:
    print("Отвергаем H₀: Конверсия в просмотр контактов выше у пользователей, кликавших на рекомендации")
else:
    print("Не отвергаем H₀: Конверсия в просмотр контактов одинакова для пользователей, кликавших на рекомендации (tips_click), и тех, кто не кликал.")

***Вывод:*** По результатам проверки конвверсия в целевое действие выше у пользователей, кликавших на рекомендации. Нулевая гипотеза отвергается.

#### Выводы:

Хотя источники трафика (Yandex/Google) не показывают различий в конверсии, взаимодействие с рекомендациями является ключевым драйвером вовлеченности. Улучшение системы рекомендаций может дать наибольший прирост конверсии - потенциально до 40% для пользователей, которые начинают взаимодействовать с этим функционалом. Рекомендуется сосредоточить ресурсы на оптимизации рекомендательной системы, а не на перераспределении маркетингового бюджета между каналами.

### Выводы и рекомендации

В рамках исследования данных о событиях и пользователях из внутренних источников приложения "Ненужные вещи" были выявлены интересные особенности и закономерности, которые могут быть использованы для разработки эффективных стратегий развития приложения.

Перед началом исследования были определены следующие задачи:

* Сегментация пользователей на основе их действий;
* Определение группы пользователей, которые чаще возвращаются в мобильное приложение (Retention rate);
* Определение группы пользователей, которые чаще совершают просмотр контактов (конверсия в целевое действие);
* Проверка гипотезы о том, что пользователи, установившие приложение из Яндекса и Google, демонстрируют разную конверсию в просмотры контактов;
* Проверка гипотезы о том, что пользователи, которые кликают по рекомендованным объявлениям, имеют более высокую конверсию в просмотр контактов, чем те, кто этого не делает.

Данные демонстрируют противоречивую динамику: при общем росте времени использования (+46%) и конверсии (до 21%) сохраняются глубинные проблемы с удержанием и вовлеченностью. Ключевая тревожная сигнализация — катастрофически низкий 7-дневный retention (6%), что втрое ниже минимальной нормы, и массовый отток новых пользователей (53.6% уходят после первого посещения). Анализ тепловых карт показывает, что однократные пользователи не возвращаются после первого дня, умеренные пользователи демонстрируют постепенное снижение удержания, а активные пользователи сохраняют наивысший уровень вовлеченности и удержания на протяжении всех семи дней. Особую озабоченность вызывают "нулевые сессии" (29% всех взаимодействий), особенно после работы с рекомендациями — основным драйвером конверсии для вовлеченной аудитории.

Хотя средние показатели улучшаются, их рост обеспечен исключительно притоком новых пользователей, маскирующим снижение глубины вовлечения. За внешне позитивными трендами скрывается опасная зависимость от "одноразовой" аудитории, в то время как реальную ценность создает узкая прослойка лояльных пользователей (15%), демонстрирующих выдающиеся метрики: их конверсия в 3 раза выше, а удержание — в 5 раз лучше, чем у основной массы.

По поводу ***рекомендаций*** хочется сказать что главная возможность роста лежит не в привлечении новой аудитории, а в качественном преобразовании существующего потока. Критически важно сосредоточиться на "переходе" пользователей от первого знакомства к повторному взаимодействию — уже вторая сессия даст почти двукратный рост конверсии и сформирует зачатки лояльности. Приоритетными направлениями должны стать: упрощение первого взаимодействия, исправление *визуальных* функций (рекомендации, поиск, просмотр фото), и стимулирование повторных визитов через персонализированные триггеры.

Этот комплекс мер позволит превратить мимолетный интерес в устойчивую привычку использования, раскрыв потенциал приложения, который уже подтвержден лояльными пользователями и позитивной реакцией на рабочие функции.